

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [31]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  x = x.to(device)
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [32]:


class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.gru = torch.nn.GRU(N + 1, hidden_size, num_layers=1, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, N + 1)
    self.hidden = None

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    x, _ = self.gru(x)
    x = self.fc(x)
    return x






  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z.
    # You need to map those characters to one-hot encodings,
    # then get the result from your network, and then convert the output
    # back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.
    # TODO
      idx = [(ord(c) - ord('a') +1) for c in s]
      idx = torch.tensor(idx, dtype=torch.int64)
      onehot = idx_to_onehot(torch.tensor(idx, dtype=torch.int64)).unsqueeze(0)
      logits = self.forward(onehot)
      pred = torch.argmax(logits, dim=2)
      pred = pred.squeeze(0)
      pred_str = ''.join([chr(c + ord('a') - 1) for c in pred])
      return pred_str






## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [33]:
D = 4
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D

  return correct / total

In [36]:
import time
start_time = time.time()

# TODO
model = GRUMemory(hidden_size=128)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_fn = torch.nn.CrossEntropyLoss()

#TODO
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)
        x_onehot, y_onehot = idx_to_onehot(x), idx_to_onehot(y)

        optimizer.zero_grad()
        logits = model(x_onehot)[:, 4:, :]
        y = y[:, 4:]
        loss = criterion(logits.reshape(-1, logits.size(-1)), y.reshape(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
    return loss.item()

def main_training_loop(model, optimizer, criterion, scheduler, DELAY, DATASET_SIZE, device):
    epoch = 0
    test_acc = 0
    while test_acc <= 0.99:
        ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)
        dataloader = torch.utils.data.DataLoader(ds, batch_size=512)

        loss = train_epoch(model, dataloader, optimizer, criterion, device)
        test_acc = test_model(model)
        scheduler.step()

        print(f'epoch {epoch}: {test_acc}, loss: {loss}')
        epoch += 1


# if torch.cuda.is_available():
#     model = model.cuda()
#     loss_fn = loss_fn.cuda()
#     device = torch.device("cuda:0")
# else:
device = torch.device("cpu")

main_training_loop(model, optimizer, loss_fn, scheduler, DELAY, DATASET_SIZE, device)


end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

<ipython-input-32-7d9216bcec0b>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  onehot = idx_to_onehot(torch.tensor(idx, dtype=torch.int64)).unsqueeze(0)


epoch 0: 0.9997476659096644, loss: 0.001718042534776032
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [39]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [68]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    #TODO
    self.max_delay = max_delay
    self.gru  = torch.nn.GRU(hidden_size, hidden_size, batch_first=True)
    self.fc = torch.nn.Linear(hidden_size, N + 1)
    self.embed = torch.nn.Embedding(N + 1, hidden_size//2)
    self.delay_embed = torch.nn.Embedding(max_delay + 1, hidden_size//2)


  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

    # TODO
    x = self.embed(x)
    delays = self.delay_embed(delays).unsqueeze(1)
    delays = delays.repeat(1, x.shape[1], 1)
    x = torch.concat((delays, x), dim=2)
    x, _ = self.gru(x)
    x = self.fc(x)
    return x

  @torch.no_grad()
  def test_run(self, s, delay):
    device = next(self.parameters()).device  # 获取模型参数所在的设备
    idx = [(ord(c) - ord('a') + 1) for c in s]
    idx = torch.tensor(idx, dtype=torch.int64).to(device)  # 转移到相应的设备
    idx = idx.unsqueeze(0)
    delay = torch.tensor(delay, dtype=torch.int64).to(device)  # 转移到相应的设备
    delay = delay.unsqueeze(0)
    self.eval()
    with torch.no_grad():
        logits = self.forward(idx, delay)

    pred = torch.argmax(logits, dim=2)
    pred = pred.squeeze(0)
    pred_str = ''.join([' ' if c == 0 else chr(c + ord('a') - 1) for c in pred])

    return pred_str



### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [69]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [70]:
model == model.cuda()

True

In [74]:
import time
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = VariableDelayGRUMemory(hidden_size=512, max_delay=MAX_DELAY).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 200, 250], gamma=0.8)
loss_fn = torch.nn.CrossEntropyLoss(reduction='none').to(device)

epoch = 0
acc = 0
while acc <= 0.9935:
    dataset = VariableDelayEchoDataset(max_delay=8, seq_length=20, size=2000)
    train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=64)
    for x, delay, y in train_dataloader:
        x, delay, y = x.to(device), delay.to(device), y.to(device)
        model.train()
        optimizer.zero_grad()
        logits = model(x, delay)
        mask = [torch.zeros(d).to(device) for d in delay]
        mask.append(torch.zeros(SEQ_LENGTH).to(device))
        mask = torch.nn.utils.rnn.pad_sequence(mask, padding_value=1, batch_first=True)[:-1]

        logits = logits.reshape(-1, logits.size(-1))

        loss = loss_fn(logits, y.reshape(-1))
        loss = loss.reshape(-1, SEQ_LENGTH)
        loss = torch.mean(loss * mask, dim=1)
        loss = loss.mean()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.0005)

        optimizer.step()

    model = model.cuda()
    acc = test_variable_delay_model(model)
    print(f'epoch {epoch}: {acc}', f'loss: {loss.item()}')
    epoch += 1

end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

epoch 0: 0.24209337349397592 loss: 1.612410545349121
epoch 1: 0.42842767295597484 loss: 1.2585194110870361
epoch 2: 0.4831758034026465 loss: 1.0064748525619507
epoch 3: 0.5418841660452399 loss: 0.9660218954086304
epoch 4: 0.5925558312655087 loss: 0.9368444681167603
epoch 5: 0.6420132906719173 loss: 0.6933175921440125
epoch 6: 0.6466625077978789 loss: 0.8243462443351746
epoch 7: 0.693759250123335 loss: 0.6743534207344055
epoch 8: 0.6914747977598009 loss: 0.7727882862091064
epoch 9: 0.7091478696741854 loss: 0.5759623050689697
epoch 10: 0.6986440248384235 loss: 0.5656790733337402
epoch 11: 0.7329823241820234 loss: 0.5521296262741089
epoch 12: 0.7577647499064488 loss: 0.5682928562164307
epoch 13: 0.7552938228292194 loss: 0.5061231851577759
epoch 14: 0.7792854475289431 loss: 0.3812147080898285
epoch 15: 0.7791265173319984 loss: 0.5700982213020325
epoch 16: 0.7982673267326733 loss: 0.6252917647361755
epoch 17: 0.8093178447849728 loss: 0.4380539059638977
epoch 18: 0.7827294533518495 loss: 0.3